<a href="https://colab.research.google.com/github/EvelynL09/data-mining/blob/master/prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install surprise

     |████████████████████████████████| 6.5MB 2.6MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1678238 sha256=f7c6259c5bfa8571cf5a63e132b6eeb0c3efee1d5a965a48a5cfd064e5579cf7
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [0]:
import sklearn
import surprise
import numpy as np
import pandas as pd

In [5]:
movies_raw = pd.read_csv('data/movies.csv')

FileNotFoundError: ignored

In [0]:
#unused tags_raw = pd.read_csv('data/genome-tags.csv')

In [0]:
#unused scores_raw = pd.read_csv('data/genome-scores.csv')

In [0]:
val_raw = pd.read_csv('data/val_ratings_binary.csv')

In [0]:
test = pd.read_csv('data/test_ratings.csv')

In [0]:
movies = movies_raw[['movieId','genres']]

In [0]:
def separateGenres(df_old):
    df_new = pd.DataFrame(columns = ['mid','genre'])
    ind = 0
    end = len(df_old.index)
    while ind < end:
        id = df_old['movieId'][ind]
        gen_list = df_old['genres'][ind].split('|')
        data = {'mid': np.repeat(id,len(gen_list)), 'genre': gen_list}
        df_add = pd.DataFrame(data)
        df_new = pd.concat([df_new,df_add],ignore_index=True)
        ind += 1
    return df_new

In [0]:
genres = separateGenres(movies)


In [0]:
genres.to_csv('genres.csv')

In [0]:
from surprise.dataset import Dataset
from surprise.reader import Reader
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold, PredefinedKFold
from sklearn.metrics import roc_auc_score

In [0]:
reader = Reader(rating_scale=[0,1],skip_lines=1,sep=',') #set params for reading in file

In [0]:
data = Dataset.load_from_file('data/train_ratings_binary.csv',reader)

In [0]:
#use cross validation to come up with AUC score in order to select best model
# nfold = 2
# kf = KFold(n_splits = nfold,shuffle=True)
# auc = []

# model = SVD()

# for trainset, testset in kf.split(data):
#     model.fit(trainset)
#     pred = model.test(testset)

#     ytrue = [x[2] for x in testset]
#     ypred = [int(round(pred[x].est)) for x in range(len(pred))]
#     print(ypred[0:100])

#     auc += [roc_auc_score(ytrue,ypred)]

# print(np.mean(auc))

[1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0]


[1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1]


0.7222007379402626


# 新段落

In [0]:
#add a column to the test df as a placeholder for ratings so that SVD alg will be happy
test_raw = pd.read_csv('data/test_ratings.csv')
ratings0 = {'rating': np.zeros(len(test_raw.index))}
df_add = pd.DataFrame(ratings0)
test_df = pd.concat([test_raw,df_add],axis=1)
test_df.to_csv('test_df.csv',index=False)

In [0]:
#fit model with trainset and predict testset values
model = SVD()
folds_files = [('data/train_ratings_binary.csv','test_df.csv')]
data = Dataset.load_from_folds(folds_files, reader=reader)
pkf = PredefinedKFold()

for trainset, testset in pkf.split(data):
    model.fit(trainset)
    pred = model.test(testset)
    ytrue = [x[2] for x in testset]

    ypred = [int(round(pred[x].est)) for x in range(len(pred))] #pred has fields with est being the estimated rating, a decimal b/n 0 and 1

    auc = roc_auc_score(ytrue,ypred)

# print(ypred)
print(auc)

In [0]:
submit

,Id,rating
0,0,1
1,1,1
2,2,1
3,3,1
4,4,0
...,...,...
4054446,4054446,1
4054447,4054447,1
4054448,4054448,1
4054449,4054449,1


In [0]:
sum(ypred)

1997757

In [0]:
submit_dict = {'Id': list(range(0,len(ypred))), 'rating': ypred}
submit = pd.DataFrame(submit_dict)
#submit = submit.set_index(pd.Index(list(range(1,len(ypred)+1)))) #trying to make index match the example file... I think taking this line out fixed the initial bug?
submit

,Id,rating
0,0,1
1,1,1
2,2,1
3,3,1
4,4,0
...,...,...
573688,573688,1
573689,573689,1
573690,573690,1
573691,573691,1


In [0]:
submit.to_csv('submit.csv',index=False)

In [0]:
tags_raw

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s
...,...,...
1123,1124,writing
1124,1125,wuxia
1125,1126,wwii
1126,1127,zombie


In [0]:
scores_raw

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675
...,...,...,...
11709763,131170,1124,0.58775
11709764,131170,1125,0.01075
11709765,131170,1126,0.01575
11709766,131170,1127,0.11450


In [0]:
#tags asssociated with each genre - multiple possible
#how many times absolutely tag appears
#proportion of each genre for each tag
#what coefficient each user gave for each genre based on their list of tags
#weights? first mean to come up with coefficient for movie
#then try assumption of more movies -> higher weight
#considering their genres??